# Libraries, functions, and globals.

In [1]:
import mysql.connector
import json
import csv
import re
import numpy as np
import pandas as pd
import math
from pandas.io.json import json_normalize
from itertools import chain

calc_field_count = 0

host = "localhost"
un = "root"
pw=""
db_name = "yelp"

fp = "C:/Users/Tolis/Documents/Data Analytics Cource/CKME136 X10/Project/data/final/profiles"
fn = "business_profiles"


def mysql_result_to_df(result, cursor):
    field_names = [i[0] for i in mycursor.description]
    return pd.DataFrame(myresult, columns=field_names)

def flatten_json(y):
    """
    Converts complex/nested JSON to table format.
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def df_to_csv(df, fp, ext=".csv", na_rep=""):
    try:
        df.to_csv(fp + ext, encoding="utf-8", header = True,\
            doublequote = True, sep=",", index=False, na_rep=na_rep)
    except Exception as e:
        print("Error: {}".format(str(e)))
        
"""
    The functions below are used to transform yelp json object literals
    to a usable value such as a count, or a boolean True/False.
"""

def yelp_json_list_format(ylist):
    result = (str(ylist).replace("'", "\""))
    result = (result.replace("True", "\"True\""))
    result = (result.replace("False", "\"False\""))
    return result
    
def get_yelp_list_keys(val):
    """
    Returns keys from a obj lit if their values evaluate to True.
    """

    result = yelp_json_list_format(val)
    true_keys = []
    try:
        result = json.loads(result)
        
        for k,v in result.items():   
            if(v != "" and v!=False and v!="False"): 
                true_keys.append(k)
        return true_keys
    except:
        return float('nan')
    
def yelp_str_to_list(val):
    """
    Splits a comma sep. string to list and returns it.
    """
    try:
        result = val.split(", ")
        for i,v in enumerate(result):
            result[i] = result[i].strip().lower()
            result[i] = re.sub(r"\s+","-", result[i])
            result[i] = re.sub(r"&","and", result[i])
        return result
    except:
        return float('nan')
    
def get_yelp_loc(row):
    city = row["city"]
    city = re.sub(r"\s+","-", city)
    return [city.lower() + "-" + row["state"].lower()]
    
def flatten_list(l):
    return list(chain.from_iterable(l))

def normalize_text(x):
    result = str(x)
    result = re.sub(r"[\[\]\"\n,'\\]", "", result)
    result = re.sub(r"\s{2,}", " ", result)
    result = result.lower().strip()
    return result

# Load business table and flatten attributes field

### Connect to mysql and create df

In [2]:
mydb = mysql.connector.connect(
  host=host,
  user=un,
  passwd=pw,
  database=db_name
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM business")

myresult = mycursor.fetchall()

business_df = mysql_result_to_df(myresult, mycursor)

#Stars gets imported as a string, so convert to decimal
business_df["stars"] = business_df["stars"].apply(lambda val: float(val))

print(business_df.info())

mycursor.close()
mydb.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Data columns (total 14 columns):
business_id     192609 non-null object
name            192609 non-null object
address         192609 non-null object
city            192609 non-null object
state           192609 non-null object
postal_code     192609 non-null object
latitude        192609 non-null float64
longitude       192609 non-null float64
stars           192609 non-null float64
review_count    192609 non-null int64
is_open         192609 non-null int64
attributes      192609 non-null object
categories      192609 non-null object
hours           192609 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 20.6+ MB
None


### Flatten and transform fields

In [3]:
business_attributes = business_df["attributes"]
business_attributes_json = []

for r in business_attributes:
    r_json = json.loads(r)
    business_attributes_json.append(r_json)

business_attr_flat = json_normalize(business_attributes_json)


"""
Declare calculated fields for json object literal provided by yelp.
These object literals contain multiple keys that map to boolean values (for the most part). 

Example:

Field: BusinessParking
Value: "{'garage': True, 'street': False, 'validated': False, 'lot': True, 'valet': False}"

Transforming these objects to a list of categores, a single number (count), or a boolen field makes them easier to analyze.
Example Transfromation of BusinessParking using count:

Field: BusinessParking_Count
value: 2 (count all True values)
"""

business_attr_flat["CALC_BusinessParking_Options"] = business_attr_flat["BusinessParking"].apply(get_yelp_list_keys)
business_attr_flat["CALC_GoodForMeal_Options"] = business_attr_flat["GoodForMeal"].apply(get_yelp_list_keys)
business_attr_flat["CALC_Music_Options"] = business_attr_flat["Music"].apply(get_yelp_list_keys)
business_attr_flat["CALC_BestNights_Options"] = business_attr_flat["BestNights"].apply(get_yelp_list_keys)
business_attr_flat["CALC_Ambience_Options"] = business_attr_flat["Ambience"].apply(get_yelp_list_keys)
business_attr_flat["CALC_DietaryRestrictions_Options"] = business_attr_flat["DietaryRestrictions"].apply(get_yelp_list_keys)
business_attr_flat["CALC_HairSpecializesIn_Options"] = business_attr_flat["HairSpecializesIn"].apply(get_yelp_list_keys)


transformed_fields = ["BusinessParking", 
                      "GoodForMeal",
                      "Music",
                      "BestNights",
                      "Ambience",
                      "DietaryRestrictions",
                      "HairSpecializesIn"]

calculated_fields = ["CALC_BusinessParking_Options",
                   "CALC_GoodForMeal_Options",
                   "CALC_Music_Options",
                   "CALC_BestNights_Options",
                   "CALC_Ambience_Options",
                   "CALC_DietaryRestrictions_Options",
                   "CALC_HairSpecializesIn_Options"]


print("Transformed:",", ".join(transformed_fields))


Transformed: BusinessParking, GoodForMeal, Music, BestNights, Ambience, DietaryRestrictions, HairSpecializesIn


### Drop fields that have been transformed and concat with original df

In [4]:
business_attr_flat = business_attr_flat.drop(transformed_fields, axis=1)


business_df = pd.concat([business_df,business_attr_flat], axis=1)

#Create calculate field here instead of above to keep it at the end of the dataframe
business_df["CALC_DaysOpen"] = business_df["hours"].apply(get_yelp_list_keys)
business_df["CALC_Categories"] = business_df["categories"].apply(yelp_str_to_list)
business_df["CALC_Location"] = business_df.apply(get_yelp_loc, axis=1)

calculated_fields.append("CALC_DaysOpen")
calculated_fields.append("CALC_Categories")
calculated_fields.append("CALC_Location")

business_df = business_df.drop(["hours"], axis=1)
business_df = business_df.drop(["attributes"], axis=1)
business_df = business_df.drop(["categories"], axis=1)

#All things consisdered as possible nulls converted to actual nulls
business_df = business_df.replace(r'^\s*$', np.nan, regex=True)
business_df = business_df.replace(r'^{}*$', np.nan, regex=True)

# Flatten fields that contain list elements to obtain them as features for the binary matrix
### These fields have been prefixed with 'CALC_' and are calculated fields that convert json objects to lists

In [5]:
# The binary matrix will contain a column for each unique feature found.
# The rows will represent businesses, and each cell will be 1/0 indicating if that business has this feature.



# Obtain new df of just the calculated fields from the bussiness df
calc_field_count = len(calculated_fields)
calc_fields = list(business_df.columns)
calc_fields = calc_fields[-calc_field_count:]
df_calc_fields = business_df[calc_fields]

# Stores all unique values found in calculated columns
bin_cols = []

# Stores a mapping table that assists in the logic for the creation of the binary matrix
field_map = []

# Loop through each calculated column
for name, values in df_calc_fields.iteritems():
    
    # Remove null rows from calc column since they are not needed for the bin matrix
    reduced = pd.Series(values)
    no_nulls = reduced.notnull()
    reduced = reduced[no_nulls]
    
    # Convert values in calc column (which are lists) to string
    # Doing this makes it possible to obtain unique values from this column
    reduced = reduced.astype(str)
    reduced = reduced.unique().tolist()
    
    # Append unique values found to bin_cols
    bin_cols.append(reduced)


# Loop through bin_cols values which are strings
# The goal is to split the strings into a list, 
# and only obtain unique single elements
for i,v in enumerate(bin_cols):
   
    # Remove any uneeded chars to obtain only values
    bin_cols[i] = normalize_text(v)
    
    # Split result to list by space and obtain unique values.
    # The normalization of text ensures text that had spaces was replaced with hyphens
    bin_cols[i] = bin_cols[i].split(" ")
    
    # convert to series to obtain unique elements easily
    bin_cols[i] = pd.Series(bin_cols[i])
    bin_cols[i] = bin_cols[i].unique()
    
    # append current field_name and unique count to field_map
    # which will be used as an index table to keep values together with what field they came from
    field_name = calculated_fields[i]
    val_count = len(bin_cols[i])
    field_map.append([field_name, val_count])

# Flatten bin cols to remove nested lists
bin_cols = flatten_list(bin_cols)

print(bin_cols[0:10])


['lot', 'garage', 'valet', 'street', 'validated', 'lunch', 'dinner', 'brunch', 'breakfast', 'dessert']


In [6]:
# The following script will create the index start and end columns for the map table.
# This makes it easy to obtain slices of the bin_cols list which creating the bool results below.
field_map = pd.DataFrame(field_map, columns=["Field Name", "Unique Count"])

#defaults
start_at = 0
end_at = 0
result = []

for i,v in field_map["Unique Count"].items():
    if(end_at == 0):
        end_at = v - 1
    else:
        start_at = end_at + 1
        end_at = v + start_at - 1
    result.append([start_at, end_at])

result = pd.DataFrame(result, columns=["Index Start","Index End"])
field_map = pd.concat([field_map, result], axis=1)
print(field_map)

                         Field Name  Unique Count  Index Start  Index End
0      CALC_BusinessParking_Options             5            0          4
1          CALC_GoodForMeal_Options             6            5         10
2                CALC_Music_Options             6           11         16
3           CALC_BestNights_Options             7           17         23
4             CALC_Ambience_Options             9           24         32
5  CALC_DietaryRestrictions_Options             6           33         38
6    CALC_HairSpecializesIn_Options             8           39         46
7                     CALC_DaysOpen             7           47         53
8                   CALC_Categories          1301           54       1354
9                     CALC_Location          1126         1355       2480


## Create bin matrix and save to csv file for later use

#### *This script may take serveral minutes to complete.

In [7]:
bin_data = []

# loop through rows in calculated fields
for i,row in df_calc_fields.iterrows():
    
    # Stores bool results from checking if features exist for current item
    bools = []
    
    # Loop through each field_name and value in current row.
    for k,v in row.items():   
        # Finds current row in field map
        which_field = field_map["Field Name"] == k
        
        # Get start and end cords for slicing the bin_cols list
        start = int(field_map["Index Start"][which_field])
        end = int(field_map["Index End"][which_field]) + 1
        
        # Slice bin cols list
        # This contains the list of boolean features to look for in the current field
        feat_values = bin_cols[start:end]
        
        # Make sure current value is a list
        if(isinstance(v, list) and len(v)>0):
            
            # Series created to utilize the isin method
            tmp_series = pd.Series(feat_values)
            
            # Normalize text before running the bool checks
            norm_vals = list(map(normalize_text, v))
            
            # Bool check for features in current field
            bool_result = tmp_series.isin(norm_vals).tolist()
           
           
            bools.append(bool_result)
            
        else:
            #if not a list then append all false values
            bools.append([False]*len(feat_values))
    
    # Flatten bools to treat all checks as a single row 
    bools = flatten_list(bools)
    # Append row for the bin matrix
    bin_data.append(bools)

In [8]:
# Convert bin matrix to data frame and transform True/False to 1/0
df_bin_data = pd.DataFrame(bin_data, columns=bin_cols)
df_bin_data = df_bin_data.astype(int)

# Drop features that have less than 3 business that contain their value's
# This is to reduce the column size as well as to remove features that don't add much to the model
bin_data_col_sums = df_bin_data.sum(axis=0)
which_field = bin_data_col_sums < 3
drop_index = bin_data_col_sums[which_field].index.tolist()

df_bin_data = df_bin_data.drop(drop_index, axis=1)

# Amount of columns to drop
print(len(drop_index))

677


In [9]:
df_bin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Columns: 1804 entries, lot to litchfield-az
dtypes: int32(1804)
memory usage: 1.3 GB


In [10]:
# Insert business id to label each row
df_bin_data.insert (0, "business_id", business_df["business_id"])


print(df_bin_data.shape)

(192609, 1805)


In [11]:
df_bin_data.head()

,business_id,lot,garage,valet,street,validated,lunch,dinner,brunch,breakfast,...,oberlin-oh,pheonix-az,cuyahoga-fls-oh,walton-hills-oh,highland-hills-oh,tottenham-on,fairport-harbor-oh,russellton-pa,mcadenville-nc,litchfield-az
0,--1UhMGODdWsrMastO9DZw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,--6MefnULPED_I942VcFNA,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,--7zmmkVg-IMGaXbuVd0SQ,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,--8LPVSo5i0Oo61X01sV9A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,--9QQLMTbFzLJ_oT-ON3Xw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Output to csv for later use.
df_to_csv(df_bin_data, fp + "/" + fn)

In [ ]:
#Clean Up
business_df = None
df_bin_data = None